In [1]:
import os
import copy
import torch
from omegaconf import OmegaConf

from src.nn.nn_dataset import DataSampler
from src.ode.sm_models_d import SynchronousMachineModels
from src.nn.nn_actions import NeuralNetworkActions


In [2]:
# Device detection
def detect_device():
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    print(f"Using device: {device}")
    return device


device = detect_device()

# Load configuration and apply overrides for fast debugging
cfg = OmegaConf.load("src/conf/setup_dataset_nn.yaml")

# Debug-friendly settings
cfg.nn.type = "DynamicNN"
cfg.nn.lr = 1e-3
cfg.nn.num_epochs = 20
cfg.nn.early_stopping = False

# Reduce dataset size for quick tests
cfg.dataset.perc_of_data_points = 0.05
cfg.dataset.perc_of_col_points = 0.05

# Weighting scheme being tested
cfg.nn.weighting.update_weights_freq = 5
cfg.nn.weighting.update_weight_method = "Static"   # or "Static", "ID", "DN", "WB", "Sam"

# Dataset path
dataset_path = "data/SM_AVR_GOV/dataset_set5_mixed.pkl"
assert os.path.exists(dataset_path), f"Missing dataset: {dataset_path}"

Using device: mps


In [3]:
# Load data + model physics
ds = DataSampler(cfg, dataset_path=dataset_path)
modelling_full = SynchronousMachineModels(cfg)

# Create NeuralNetworkActions instance
net = NeuralNetworkActions(cfg, modelling_full, data_loader=ds)
# Call pinn_train2 EXACTLY how it was designed
net.pinn_train2(
    num_of_skip_data_points=1,
    num_of_skip_col_points=1,      # correct for new code
    num_of_skip_val_points=1,
    wandb_run=None,
)

# Print summary
print("\nTraining completed.")
print(f"Final total loss: {net.loss_total.item():.4e}")
print(f"Data loss:        {net.loss_data.item():.4e}")
print(f"dt loss:          {net.loss_dt.item():.4e}")
print(f"PINN loss:        {net.loss_pinn.item():.4e}")
print(f"IC loss:          {net.loss_pinn_ic.item():.4e}")

SM_AVR_GOV data


/Users/jonaswiendl/local/02456_DeepLearning/src/nn/nn_dataset.py:186: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  training_sample = torch.tensor(training_sample, dtype=torch.float32) # convert the trajectory to tensor


Number of training samples:  800000 Number of validation samples:  100000 Number of testing samples:  100000
Number of different initial conditions for collocation points:  100
['theta', 'omega', 'E_d_dash', 'E_q_dash', 'R_F', 'V_r', 'E_fd', 'P_sv', 'P_m'] Variables
[[-2, 2], [-1, 1], [0], [1], [1], [1.105], [1.08], [0.7048], [0.7048]] Set of values for init conditions
[10, 10, 1, 1, 1, 1, 1, 1, 1] Iterations per value
Selected deep learning model:  DynamicNN
Number of labeled training data: 40000 Number of collocation points: 5000 Number of collocation points (IC): 100 Number of validation data: 100000
Weights initialized as:  [1, 0.001, 0.0001, 0.001]  are updated with scheme:  Static 
getting in training
Model( and tf values) saved: model/data_dt_pinn_ic/SM_AVR_GOVDynamicNN_1_20_40000_5000_100000_None_None_1_0.001_0.0001_0.001_Static.pth
Total test trajectories 100
Loss: 0.00244825
MAE Loss: 0.02837128
Total trainable parameters 17929

Training completed.
Final total loss: 6.5812e-0